In [4]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [5]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo64'
DB_PASSWORD = 'grupo64'
DB_NAME = 'grupo64e2'

In [6]:

# Ruta del archivo CSV
archivo_csv = '../datos e2/clientes.csv'

# Configuración de la conexión
try:
    conexion = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print(conexion)
    # Crear un cursor
    cursor = conexion.cursor()
    print(cursor)

    # Crear la tabla (ajusta esto según tu esquema de datos)
    crear_tabla_query = """
    CREATE TABLE IF NOT EXISTS clientes (
        nombre varchar   NOT NULL,
        email varchar   NOT NULL,
        telefono varchar   NOT NULL,
        clave varchar       NOT NULL,
        direccion varchar   NOT NULL,
        comuna varchar    NOT NULL,
        CONSTRAINT pk_Cliente PRIMARY KEY (
            email))
    """
    cursor.execute(crear_tabla_query)
    conexion.commit()

    # Preparar el query de inserción
    insercion_query = """
    INSERT INTO clientes (nombre, email, telefono, clave, direccion, comuna)
    VALUES (%s, %s, %s, %s, %s, %s);
    """

    # Leer y procesar el archivo CSV
    with open(archivo_csv, 'r') as file:
        # Leer la primera línea para obtener los nombres de las columnas
        columnas = file.readline().strip().split(';')
        print(columnas)

        id_count = 0
        # Iterar sobre cada línea en el archivo CSV
        for linea in file:
            # Dividir la línea en valores
            valores = linea.strip().split(';')
            print(valores)

            try:
                cursor.execute(insercion_query, (valores[0], valores[1], valores[2], valores[3], valores[4], valores[5]))
            except Error as e:
                    print(f"Error al insertar la línea {linea}: {e}")
                    conexion.rollback()
                    continue

    # Confirmar la inserción
    conexion.commit()

except Error as e:
    print(f"Error de conexión o durante la operación en la base de datos: {e}")

finally:

    if cursor:
        cursor.close()
    if conexion:
        conexion.close()

<connection object at 0x000001CD34877340; dsn: 'user=grupo64 password=xxx dbname=grupo64e2 host=pavlov.ing.puc.cl port=5432', closed: 0>
<cursor object at 0x000001CD3486B760; closed: 0>
['cliente', 'email', 'telefono', 'clave', 'direccion', 'comuna']
['Agustin Matte', 'Agustin.Matte@bbdduc.utopia', '56911904638', 'JQMUGFRS73', 'Calle Estado 456, Santiago Centro', '13101']
['Agustin Matte', 'Agustin.Matte@bbdduc.utopia', '56911904638', 'JQMUGFRS73', 'Av. Alemania 456, Temuco', '9101']
Error al insertar la línea Agustin Matte;Agustin.Matte@bbdduc.utopia;56911904638;JQMUGFRS73;Av. Alemania 456, Temuco;9101
: duplicate key value violates unique constraint "pk_cliente"
DETAIL:  Key (email)=(Agustin.Matte@bbdduc.utopia) already exists.

['Agustin Reyes', 'Agustin.Reyes@bbdduc.utopia', '56972317149', 'SFCGUHRR74', 'Calle Prat 789, Iquique', '1101']
['Agustin Reyes', 'Agustin.Reyes@bbdduc.utopia', '56972317149', 'SFCGUHRR74', 'Av. Alemania 789, Temuco', '9101']
Error al insertar la línea Agust

IndexError: list index out of range